In [8]:
import os
import sys
sys.path.append(f"{os.getcwd()}/../src")

## Setup constants

In [9]:
n_classic_x = 55
m_classic_x = 40
dv_classic_x = 3


n_classic_z = 55
m_classic_z = 45
dv_classic_z = 3

n_cross_checks = 10

## Construct the Codes without Cross Checks

In [10]:
from CPC import cpc_code
from ldpc_classical import code_gen

In [11]:
Hx, Hz = code_gen.gen_code(n_classic_x, m_classic_x, dv_classic_x, 1), code_gen.gen_code(n_classic_z, m_classic_z, dv_classic_z, 2)
Hx.shape, Hz.shape

((40, 55), (45, 55))

In [12]:
cpc = cpc_code.gen_cpc_from_classical_codes(Hx, Hz)

Simplifying the input code


## Add Cross Checks

In [22]:
import random

i = 0
check_verts = []
[check_verts.append(v) for v in cpc.vertices if v.check_qubit]

sample_list = list(range(len(check_verts)))

while i < n_cross_checks:
	idxs = random.sample(sample_list, 2)
	cross_edge = cpc_code.CPCEdge(check_verts[idxs[0]], check_verts[idxs[1]])
	if not cpc.has_edge(cross_edge):
		cpc.add_edge(cross_edge)
		i += 1

cpc.simplify_virtual_edges()

## Run BP trials on the tanner graph

In [23]:
H, _ = cpc.get_tanner_graph()
n_qubits = len(H[0])

In [24]:
from ldpc_classical.aff3ct_wrapper import aff3ct_simulate

aff3ct_simulate.get_wer(H, [0.01] * n_qubits) 

0.06630000000000003